# Azure AI агенти са подршком за Model Context Protocol (MCP) - Python

Овај нотебук демонстрира како користити Azure AI агенте са алатима за Model Context Protocol (MCP) у Python-у. Показује како креирати интелигентног агента који може користити спољне MCP сервере (као што је Microsoft Learn) за побољшане могућности користећи аутентификацију без кључа.


## Инсталирајте потребне Python пакете

Прво, потребно је да инсталирамо неопходне Python пакете:
- **azure-ai-projects**: Основни SDK за Azure AI Projects
- **azure-ai-agents**: Azure AI Agents SDK за креирање и управљање агентима
- **azure-identity**: Омогућава аутентификацију без кључа користећи DefaultAzureCredential
- **mcp**: Имплементација Model Context Protocol за Python


## Предности аутентикације без кључа

Овај нотебук демонстрира **аутентикацију без кључа**, која пружа неколико предности:
- ✅ **Нема потребе за управљањем API кључевима** - Користи аутентикацију засновану на Azure идентитету
- ✅ **Побољшана безбедност** - Нема чувања тајни у коду или конфигурационим фајловима
- ✅ **Аутоматска ротација акредитива** - Azure управља животним циклусом акредитива
- ✅ **Контрола приступа заснована на улогама** - Користи Azure RBAC за детаљну контролу дозвола
- ✅ **Подршка за више окружења** - Ради беспрекорно у развојном и продукционом окружењу

`DefaultAzureCredential` аутоматски бира најбољи доступни извор акредитива:
1. **Managed Identity** (приликом покретања у Azure-у)
2. **Azure CLI** акредитиви (током локалног развоја)
3. **Visual Studio** акредитиви
4. **Променљиве окружења** (ако су конфигурисане)
5. **Аутентикација преко интерактивног претраживача** (као резервна опција)


## Подешавање аутентификације без кључа

**Предуслови за аутентификацију без кључа:**

### За локални развој:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### За Azure окружења:
- Омогућите **Системски додељен Управљани Идентитет** на вашем Azure ресурсу
- Доделите одговарајуће **RBAC улоге** управљаном идентитету:
  - `Cognitive Services OpenAI User` за приступ Azure OpenAI
  - `AI Developer` за приступ Azure AI пројектима

### Променљиве окружења (Опционо):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Нису потребни API кључеви или стрингови за повезивање!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Увоз потребних библиотека

Увезите неопходне Python модуле:
- **os, time**: Стандардне Python библиотеке за променљиве окружења и кашњења
- **AIProjectClient**: Главни клијент за Azure AI пројекте
- **DefaultAzureCredential**: Аутентикација без кључа за Azure услуге
- **MCP-related classes**: За креирање и управљање MCP алатима и руковање одобрењима


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Подешавање MCP серверских поставки

Подесите конфигурацију MCP сервера користећи променљиве окружења са подразумеваним вредностима:
- **MCP_SERVER_URL**: URL MCP сервера (подразумевано Microsoft Learn API)
- **MCP_SERVER_LABEL**: Ознака за идентификацију MCP сервера (подразумевано "mslearn")

Овај приступ омогућава флексибилну конфигурацију у различитим окружењима.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Креирање Azure AI Project клијента (Аутентикација без кључа)

Иницијализујте Azure AI Project клијента користећи **аутентикацију без кључа**:
- **endpoint**: URL крајње тачке Azure AI Foundry пројекта
- **credential**: Користи `DefaultAzureCredential()` за безбедну аутентикацију без кључа
- **Нису потребни API кључеви**: Аутоматски открива и користи најбоље доступне акредитиве

**Ток аутентикације:**
1. Проверава Managed Identity (у Azure окружењима)
2. Прелази на Azure CLI акредитиве (за локални развој)
3. Користи друге доступне изворе акредитива по потреби

Овај приступ елиминише потребу за управљањем API кључевима или конекционим стринговима у вашем коду.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Креирај MCP дефиницију алата

Креирај MCP алат који се повезује са Microsoft Learn MCP сервером:
- **server_label**: Идентификатор за MCP сервер
- **server_url**: URL крајња тачка MCP сервера
- **allowed_tools**: Опциона листа за ограничавање којим алатима се може приступити (празна листа дозвољава све алате)

Овај алат ће омогућити агенту приступ Microsoft Learn документацији и ресурсима.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Креирање агента и извршавање разговора (рад без кључа)

Овај свеобухватни одељак приказује комплетан **радни процес агента без кључа**:

1. **Креирајте AI агента**: Поставите агента са GPT-4.1 nano моделом и MCP алатима
2. **Креирајте нит**: Успоставите нит разговора за комуникацију
3. **Пошаљите поруку**: Питајте агента о разликама између Azure OpenAI и OpenAI
4. **Руковање одобрењима алата**: Аутоматски одобрите позиве MCP алата када је потребно
5. **Пратите извршење**: Надгледајте напредак агента и обавите све потребне радње
6. **Прикажите резултате**: Прикажите детаље разговора и употребе алата

**Карактеристике рада без кључа:**
- ✅ **Без фиксних тајни** - Сва аутентификација се обавља преко Azure идентитета
- ✅ **Безбедност по подразумеваном подешавању** - Користи контролу приступа засновану на улогама
- ✅ **Олакшано постављање** - Нема потребе за управљањем акредитивима
- ✅ **Погодно за ревизију** - Сваки приступ се прати преко Azure идентитета

Агент ће користити MCP алате за приступ ресурсима Microsoft Learn уз потпуну безбедност и без управљања API кључевима.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да превод буде тачан, молимо вас да имате у виду да аутоматизовани преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
